# Tool to scrap tweets using _Twitter API v2_

This is a tool to search for tweets with specific keywords. 
To use this tool, is necessary to create a project app in Twitter as [developer](https://developer.twitter.com/)

In [ ]:
import requests
import os
import json

##Token from API

In [ ]:
BEARER_TOKEN = "Type in your Key here"

##Search function

To search for the keywords adidas, the query was built based on the  *Twitter API v2* that can be reviewed [here](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#boolean)


In [ ]:
#define search twitter function
def search_twitter(query, tweet_fields, start_time, end_time, max_results, bearer_token = BEARER_TOKEN):
  headers = {"Authorization": "Bearer {}".format(bearer_token)}

  url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&{}&{}&{}".format(query, tweet_fields,start_time,end_time,max_results)    #query formating
  response = requests.request("GET", url, headers=headers)
  print(url)
  print(response.status_code)

  if response.status_code != 200:
    raise Exception(response.status_code, response.text)
  return response.json()

##Query parameters

The query was focused with positive, negative, buy and sell key word, to improve the context of the tweets.

###Positive sentiments query

In [ ]:
#positive query
queryP = "adidas (happy OR exciting OR excited OR favorite OR fav OR amazing OR lovely OR incredible OR is not) -horrible -worst -sucks -bad -disappointing -hate -despite -dislike -awful lang:en -is:retweet " #twitter fields to be returned by api call


###Negative sentiment query

In [ ]:
#negative query
queryN ="adidas (horrible OR worst OR sucks OR bad OR disappointing OR hate OR despite OR dislike OR awful) -happy -exciting -excited -favorite -fav -amazing -lovely -incredible -is not -bunny lang:en -is:retweet"


###Intent

In [ ]:
queryBuy = "adidas (will OR buy OR get OR purchase OR now OR tomorrow OR acquire OR will have OR go for OR endure OR charter OR bargain OR bought OR invest OR need OR want OR will get) -horrible -worst -sucks -bad -disappointing -sign up now -SALE -Clearance -selling -shop up -auction lang:en -is:retweet " #twitter fields to be returned by api call
querySell = "adidas (sign up now OR SALE OR clearance OR selling OR free shiping OR discount OR auction OR shop up) -horrible -worst -sucks -bad -disappointing  lang:en -is:retweet " #twitter fields to be returned by api call




---



##Additional Quiery parameters

In [ ]:
tweet_fields = "tweet.fields=text,created_at"
max_results="max_results=100"

#Execute query

In [ ]:
import numpy as np
import time

Based on the dates the query was done (only 7 days windows allowed by the Standard product track at the [Basic access level](https://developer.twitter.com/en/products/twitter-api/early-access/guide#na_1))

In [ ]:
hours = np.arange(0,25,1)
days = [30, 31, 1, 2, 3, 4, 5] 

In [ ]:
dataP = []
dataN = []
dataBuy = []
dataSell = []
start_time = ''
end_time=''
i=0
for d in days:

  if d == 3:
    print('Waintg for 15min  ')
    time.sleep(900)

  h=0
  while h < 24:
    nxh = h+1
   
    if d >= 1 and d <= 5:
      if nxh == 24:
        end_time1="end_time=2021-09-"+str(d)+"T23:59:00.000Z"
      else:
        start_time1="start_time=2021-09-"+str(d)+"T"+str(h)+":00:00.000Z"
        end_time1="end_time=2021-09-"+str(d)+"T"+str(nxh)+":00:00.000Z"
    else:
      if nxh == 24:
        end_time1="end_time=2021-08-"+str(d)+"T23:59:00.000Z"
      else:
        start_time1="start_time=2021-08-"+str(d)+"T"+str(h)+":00:00.000Z"
        end_time1="end_time=2021-08-"+str(d)+"T"+str(nxh)+":00:00.000Z"
    h+=1

    json_responseP = search_twitter(query=queryP, tweet_fields=tweet_fields, start_time=start_time1, end_time=end_time1,max_results=max_results, bearer_token=BEARER_TOKEN)
    json_responseN = search_twitter(query=queryN, tweet_fields=tweet_fields, start_time=start_time1, end_time=end_time1,max_results=max_results, bearer_token=BEARER_TOKEN)
    json_responseBuy = search_twitter(query=queryBuy, tweet_fields=tweet_fields, start_time=start_time1, end_time=end_time1,max_results=max_results, bearer_token=BEARER_TOKEN)
    json_responseSell = search_twitter(query=querySell, tweet_fields=tweet_fields, start_time=start_time1, end_time=end_time1,max_results=max_results, bearer_token=BEARER_TOKEN)
    dataP.append(json_responseP)
    dataN.append(json_responseN)
    dataBuy.append(json_responseBuy)
    dataSell.append(json_responseSell)
    i+=1

    if d == 5 and h == 22:
      break



https://api.twitter.com/2/tweets/search/recent?query=adidas (happy OR exciting OR excited OR favorite OR fav OR amazing OR lovely OR incredible OR is not) -horrible -worst -sucks -bad -disappointing -hate -despite -dislike -awful lang:en -is:retweet &tweet.fields=text,created_at&start_time=2021-08-30T0:00:00.000Z&end_time=2021-08-30T1:00:00.000Z&max_results=100
200
https://api.twitter.com/2/tweets/search/recent?query=adidas (horrible OR worst OR sucks OR bad OR disappointing OR hate OR despite OR dislike OR awful) -happy -exciting -excited -favorite -fav -amazing -lovely -incredible -is not -bunny lang:en -is:retweet&tweet.fields=text,created_at&start_time=2021-08-30T0:00:00.000Z&end_time=2021-08-30T1:00:00.000Z&max_results=100
200
https://api.twitter.com/2/tweets/search/recent?query=adidas (will OR buy OR get OR purchase OR now OR tomorrow OR acquire OR will have OR go for OR endure OR charter OR bargain OR bought OR invest OR need OR want OR will get) -horrible -worst -sucks -bad -di

Exception: ignored

#Convert to pandas 

In [ ]:
import pandas as pd
from pandas import json_normalize

In [ ]:
df_All = pd.DataFrame()
df_P = pd.DataFrame()
df_N = pd.DataFrame()
df_Buy = pd.DataFrame()
df_Sell = pd.DataFrame()

for i in range(len(dataP)-1):
  infoP = json.loads(json.dumps(dataP[i], indent=4, sort_keys=True,ensure_ascii=True))
  infoN = json.loads(json.dumps(dataN[i], indent=4, sort_keys=True,ensure_ascii=True))
  infoBuy = json.loads(json.dumps(dataBuy[i], indent=4, sort_keys=True,ensure_ascii=True))
  infoSell = json.loads(json.dumps(dataSell[i], indent=4, sort_keys=True,ensure_ascii=True))

  try:
    df_P = df_P.append(json_normalize(infoP['data']),ignore_index = True)
    df_N = df_N.append(json_normalize(infoN['data']),ignore_index = True)
    df_Buy = df_Buy.append(json_normalize(infoBuy['data']),ignore_index = True)
    df_Sell = df_Sell.append(json_normalize(infoSell['data']),ignore_index = True)
  except Exception:
    print('No Data')
  
  #df_All = df_All.append(json_normalize(infoAll['data']),ignore_index = True)
  #print (i)
  #print(json_normalize(infoSports['data']))

No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data


In [ ]:
df_P.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  559 non-null    object
 1   id          559 non-null    object
 2   text        559 non-null    object
dtypes: object(3)
memory usage: 13.2+ KB


In [ ]:
df_P

,created_at,id,text
0,2021-08-30T00:34:05.000Z,1432139525602611207,One of my favorite goals against FC Dallas 18/...
1,2021-08-30T00:31:26.000Z,1432138856963330049,Check out my amazing son @anthemslegolife subs...
2,2021-08-30T00:22:35.000Z,1432136630698221570,Happy Birthday MJ! 🎉🎶 What was MJ best music v...
3,2021-08-30T00:15:49.000Z,1432134930193174529,proving how tough and persistent iam to the la...
4,2021-08-30T06:44:46.000Z,1432232809918345216,"@MR_JiBoLA Yes and I’m saying, as an artist of..."
...,...,...,...
559,2021-09-05T18:22:30.000Z,1434582729098776577,@JamiroquaiHQ Love those SL-72 those are my fa...
560,2021-09-05T19:59:58.000Z,1434607256981671938,@fazylatass @Rip_Rope @JasonRileyWDRB You admi...
561,2021-09-05T19:32:32.000Z,1434600355065733124,"@AntMeaux Bruh, what were you smokin? lol\n\nT..."
562,2021-09-05T19:27:17.000Z,1434599030529708034,adidas is trying to convince us these are not ...


In [ ]:
df_N.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  18 non-null     object
 1   id          18 non-null     object
 2   text        18 non-null     object
dtypes: object(3)
memory usage: 560.0+ bytes


In [ ]:
df_N

,created_at,id,text
0,2021-08-30T12:43:58.000Z,1432323206510174210,@FutbolShIrTALIA @thesefootytimes @wunderkits ...
1,2021-08-30T21:31:05.000Z,1432455858357559298,@nellsbells0 https://t.co/XbBdsveuxX I found t...
2,2021-08-31T18:03:14.000Z,1432765943025917957,@jonspannring I did! I got a W on SNKRS and a ...
3,2021-08-31T23:16:19.000Z,1432844732166819843,@adidas Do not waste your time and money shopp...
4,2021-09-01T16:05:00.000Z,1433098575580475398,@adidasGolf im an avid adidas guy. Cross brand...
5,2021-09-03T02:22:08.000Z,1433616267739602945,Rough mockup of the new gray adidas alt. Not b...
6,2021-09-04T14:11:17.000Z,1434157121143197699,@iAmPin0yy @Vintage_Utd Not bad i surpose cons...
7,2021-09-04T15:58:30.000Z,1434184101729288194,MY LOGO NOT SO BAD; CREATED IT MYSELF..\n\nFTR...
8,2021-09-05T00:58:14.000Z,1434319930279796736,It’s so disappointing that Adidas don’t make L...
9,2021-09-05T14:56:49.000Z,1434530967637483522,@thezulfo_ Same like Li-Ning. Its picking up l...


In [ ]:
df_Buy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  6 non-null      object
 1   id          6 non-null      object
 2   text        6 non-null      object
dtypes: object(3)
memory usage: 272.0+ bytes


In [ ]:
df_Sell.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  204 non-null    object
 1   id          204 non-null    object
 2   text        204 non-null    object
dtypes: object(3)
memory usage: 4.9+ KB


Labeling the queries according to the keys used.

In [ ]:
df_P['Sent'] = 'POS'
df_N['Sent'] = 'NEG'
df_Buy['Sent'] = 'BUY'
df_Sell['Sent'] = 'SELL'

In [ ]:
df_P.drop_duplicates(subset=['created_at'],inplace=True)
df_N.drop_duplicates(subset=['created_at'],inplace=True)
df_Buy.drop_duplicates(subset=['created_at'],inplace=True)
df_Sell.drop_duplicates(subset=['created_at'],inplace=True)

In [ ]:
df = df_P.append(df_N,ignore_index=True)
df = df.append(df_Buy,ignore_index=True)
df = df.append(df_Sell,ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  770 non-null    object
 1   id          770 non-null    object
 2   text        770 non-null    object
 3   Sent        770 non-null    object
dtypes: object(4)
memory usage: 24.2+ KB


In [ ]:
df.nunique()

created_at    769
id            769
text          762
Sent            4
dtype: int64

In [ ]:
df.drop_duplicates(subset=['created_at'],inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  770 non-null    object
 1   id          770 non-null    object
 2   text        770 non-null    object
 3   Sent        770 non-null    object
dtypes: object(4)
memory usage: 24.2+ KB


In [ ]:
inx=509
df[inx:inx+15]

,created_at,id,text,Sent
509,2021-09-05T12:02:38.000Z,1434487131590778880,@English_AS @robertmarawa @Cristiano To those ...,POS
510,2021-09-05T13:23:59.000Z,1434507604265521152,@slimdot3 Common sense is not common. \nYou ar...,POS
511,2021-09-05T13:15:08.000Z,1434505379178848259,My problem is that they'll never never do this...,POS
512,2021-09-05T13:07:44.000Z,1434503516748492801,@Scornpaul @Harry188_UTD @ManUnitedMedia Not s...,POS
513,2021-09-05T14:45:30.000Z,1434528120988741634,@Canedude08 @bigsteppa46 @zumundanole @CanesFo...,POS
514,2021-09-05T14:38:31.000Z,1434526363155849220,"@EnterDaDome @BiggieRothstein Yes, and she loo...",POS
515,2021-09-05T14:13:18.000Z,1434520015303323648,@Jon_Walsh @adidasoriginals @adidas Not sure S...,POS
516,2021-09-05T15:33:33.000Z,1434540211195846668,#GirlTalkZA\n\nIs there a way of online shoppi...,POS
517,2021-09-05T15:23:59.000Z,1434537802235064321,@Matytsyn Instinctively I' d guess they can re...,POS
518,2021-09-05T15:13:55.000Z,1434535269705859078,@mynamedurell @drmartens @PUMA @adidasoriginal...,POS


#Saving dataframe to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path="/content/drive/MyDrive/Colab Notebooks/EnhanceIT/Training/Exercises/Week6/W6_Project/twitter/"

In [ ]:
df.to_csv(path+'All_twitts_v02.csv')